# 모델의 저장과 복원 구현

>### Load modules

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

NumPy Version :1.18.5
TensorFlow Version :2.3.0
Matplotlib Version :3.2.2


> ### 저장 경로 설정

In [2]:
colab=True
try:
  from google.colab import drive
except:
  colab =False
if colab : 
    drive.mount('/content/drive')
    print('g-drive mounted.')
else : print('local drive.')

Mounted at /content/drive
g-drive mounted.


In [3]:
# file path: 다른 경로에 실습파일을 복사했다면, 아래 경로를 수정하세요

if colab : 
  save_dir = '/content/drive/My Drive/files/save/'
else : 
  save_dir = 'c:/files/save/'

> ### Load MNIST Data

In [4]:
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

train_data = train_data.astype(np.double)
train_data = np.expand_dims(train_data, axis=-1)
train_data = train_data / 255.0

test_data = test_data.astype(np.double)
test_data = np.expand_dims(test_data, axis=-1)
test_data = test_data / 255.0

11493376/11490434 [==============================] - 0s 0us/step


>### Create Model


In [5]:
def create_model():
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',activation='relu',input_shape=(28, 28, 1)))
  model.add(tf.keras.layers.Dropout(rate=0.4))
  model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',activation='relu'))
  model.add(tf.keras.layers.Dropout(rate=0.4))
  model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',activation='relu'))
  model.add(tf.keras.layers.Dropout(rate=0.4))
  model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(10, activation='softmax'))

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          3

>### 훈련하는 동안 체크포인트 저장하기

In [6]:
checkpoint_path = save_dir + "cp-{epoch:04d}.ckpt"

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

history = model.fit(train_data, train_labels, shuffle=True,
                 batch_size=100, epochs=10,
                 validation_data=(test_data, test_labels),
                 callbacks = [cp_callback])          

Epoch 1/10
592/600 [============================>.] - ETA: 0s - loss: 0.2616 - accuracy: 0.9184
Epoch 00001: saving model to /content/drive/My Drive/files/save/cp-0001.ckpt
600/600 [==============================] - 3s 6ms/step - loss: 0.2593 - accuracy: 0.9190 - val_loss: 0.2023 - val_accuracy: 0.9835
Epoch 2/10
594/600 [============================>.] - ETA: 0s - loss: 0.0808 - accuracy: 0.9747
Epoch 00002: saving model to /content/drive/My Drive/files/save/cp-0002.ckpt
600/600 [==============================] - 3s 5ms/step - loss: 0.0807 - accuracy: 0.9748 - val_loss: 0.1735 - val_accuracy: 0.9882
Epoch 3/10
592/600 [============================>.] - ETA: 0s - loss: 0.0590 - accuracy: 0.9820
Epoch 00003: saving model to /content/drive/My Drive/files/save/cp-0003.ckpt
600/600 [==============================] - 3s 5ms/step - loss: 0.0588 - accuracy: 0.9820 - val_loss: 0.1155 - val_accuracy: 0.9886
Epoch 4/10
599/600 [============================>.] - ETA: 0s - loss: 0.0503 - accuracy:

In [7]:
new_model = create_model()
new_model.evaluate(test_data,  test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 2.3036 - accuracy: 0.1110


[2.303555488586426, 0.11100000143051147]

In [8]:
new_model.load_weights(save_dir + 'cp-0008.ckpt')
new_model.evaluate(test_data,  test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0548 - accuracy: 0.9939


[0.054827459156513214, 0.9939000010490417]

>### 수동으로 weight 저장하기

In [9]:
# 가중치를 저장합니다
model.save_weights(save_dir + 'cp-curr.ckpt')

In [10]:
new_model = create_model()
new_model.evaluate(test_data,  test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 2.3003 - accuracy: 0.1019


[2.300262212753296, 0.10189999639987946]

In [11]:
# 가중치를 복원합니다
new_model.load_weights(save_dir + 'cp-curr.ckpt')
new_model.evaluate(test_data,  test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0495 - accuracy: 0.9944


[0.04946812242269516, 0.9944000244140625]

> ### 모델 전체를 저장하기

In [12]:
model.save(save_dir + 'my_model.h5')

In [13]:
new_model = tf.keras.models.load_model(save_dir + 'my_model.h5')
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          3

In [14]:
new_model.evaluate(test_data,  test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.0495 - accuracy: 0.0982


[0.04946812242269516, 0.0982000008225441]